In [1]:
# Administrative stuff:
import os

# For Data manipulation:
import pandas as pd
import numpy as np
import random
import re

# Nice to haves
#import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
# import googlemaps
import plotly.graph_objects as go

## dynamic text updating
#from IPython.display import Markdown as md

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from plotly import graph_objects as go
import plotly.figure_factory as ff

# Getting Data from the Cleaning File

In [3]:
%store -r air_data
%store -r air_ken
%store -r air_ken_2019
%store -r health_county_names
%store -r health_data
%store -r health_ken
%store -r health_ken_2019
%store -r health_ken_2019_clean
%store -r health_ken_2019_fm
%store -r so2
%store -r so2_county_names
%store -r so2_ken
%store -r airq_0




%store air_data
%store air_ken
%store air_ken_2019
%store health_county_names
%store health_data
%store health_ken
%store health_ken_2019
%store health_ken_2019_clean
%store health_ken_2019_fm
%store so2
%store so2_county_names
%store so2_ken
%store so2_ken_f
%store good_days_ken
%store daily_aqi
%store daily_aqi_ken


In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Merging air and Health Data First

In [5]:
air_ken_2019.head(2)
health_ken_2019_fm.head(2)

,index,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0
1,327,Kentucky,Boone,2019,239,222,17,0,0,0,0,80,49,39,0,0,239,0,0


,index,Year,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
0,3596,2019,Adair,21001,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,44.8,"3,016",POINT (-85.15821669 37.13121962),21001970300,LPA
2,3598,2019,Anderson,21005,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,26.6,"5,372",POINT (-84.92306901 38.02049683),21005950201,DEPRESSION


In [6]:
# Initial merge: 4282432 rows × 23 columns
# Next: 6768 rows 

# Merging:

merged_ah = pd.merge(air_ken_2019, health_ken_2019_fm, how = "left", left_on = "County", right_on = "CountyName",suffixes=('_air', '_health'))
merged_ah.head(2)

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION
1,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH


In [7]:
air_ken_2019.shape
health_ken_2019_fm.shape
merged_ah.shape

(27, 19)

(13272, 13)

(6900, 32)

In [8]:
merged_ah["County"].value_counts()

Jefferson     2280
Fayette        984
Campbell       300
Warren         288
Daviess        276
Hardin         264
Boone          252
Pike           228
Christian      216
McCracken      204
Bullitt        204
Pulaski        168
Oldham         168
Boyd           156
Henderson      132
Bell           108
Jessamine      108
Greenup        108
Perry           96
Carter          84
Morgan          60
Simpson         48
Hancock         36
Edmonson        36
Trigg           36
Washington      36
Livingston      24
Name: County, dtype: int64

In [9]:
merged_ah[merged_ah["County"] == "Jefferson"]

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
3156,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,3649,2019,Jefferson,21111,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,%,Crude prevalence,6.1,"2,924",POINT (-85.78202288 38.27661851),21111000200,CANCER
3157,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,3651,2019,Jefferson,21111,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,17.2,"2,447",POINT (-85.78253076 38.2614617),21111002300,ACCESS2
3158,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,3654,2019,Jefferson,21111,Health Risk Behaviors,Current smoking among adults aged >=18 years,%,Crude prevalence,15.9,"6,253",POINT (-85.52422538 38.234524),21111010402,CSMOKING
3159,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,3655,2019,Jefferson,21111,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,21.3,"3,078",POINT (-85.64078951 38.24198879),21111010601,DEPRESSION
3160,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,825740,2019,Jefferson,21111,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,7.7,"5,222",POINT (-85.56243511 38.15331331),21111011519,ACCESS2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,841817,2019,Jefferson,21111,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,51.4,"2,540",POINT (-85.80521343 38.24337925),21111001000,LPA
5432,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,841822,2019,Jefferson,21111,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,10.9,"4,078",POINT (-85.57559354 38.25625142),21111010102,MHLTH
5433,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,841823,2019,Jefferson,21111,Prevention,Visits to doctor for routine checkup within th...,%,Crude prevalence,81.3,"6,220",POINT (-85.58511499 38.17750746),21111011110,CHECKUP
5434,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,841829,2019,Jefferson,21111,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,10.2,"1,949",POINT (-85.69251985 38.2569818),21111007601,PHLTH


In [10]:
air_ken_2019[air_ken_2019["County"] == "Bell"]
health_ken_2019_clean[health_ken_2019_clean["CountyName"] == "Bell"]

,index,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0


,index,Year,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
3,3599,2019,Bell,21013,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.8,"5,150",POINT (-83.73909778 36.76433928),21013960200,STROKE
4,3600,2019,Bell,21013,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.5,"2,381",POINT (-83.61861461 36.71356868),21013960300,STROKE
5,3601,2019,Bell,21013,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.3,"2,849",POINT (-83.72538629 36.61758576),21013960700,STROKE
191,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION
255,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13002,825231,2019,Bell,21013,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,23.3,"1,896",POINT (-83.53196181 36.73527758),21013960400,MHLTH
13103,825354,2019,Bell,21013,Health Outcomes,Diagnosed diabetes among adults aged >=18 years,%,Crude prevalence,16.9,"2,381",POINT (-83.61861461 36.71356868),21013960300,DIABETES
13152,825409,2019,Bell,21013,Health Outcomes,Coronary heart disease among adults aged >=18 ...,%,Crude prevalence,12.6,"2,564",POINT (-83.58378607 36.8540896),21013960100,CHD
13283,825572,2019,Bell,21013,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,14.1,"3,771",POINT (-83.76484451 36.64167333),21013960800,ACCESS2


In [11]:
merged_ah.head()

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION
1,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH
2,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809818,2019,Bell,21013,Health Risk Behaviors,Current smoking among adults aged >=18 years,%,Crude prevalence,33.3,"3,139",POINT (-83.81829927 36.66805338),21013961100,CSMOKING
3,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809827,2019,Bell,21013,Prevention,Visits to doctor for routine checkup within th...,%,Crude prevalence,80.7,"4,747",POINT (-83.76654247 36.59758829),21013960600,CHECKUP
4,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809884,2019,Bell,21013,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,11.8,"3,771",POINT (-83.76484451 36.64167333),21013960800,CASTHMA


In [12]:
so2_ken.head(2)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
97694,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-01,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01
97695,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-02,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01


In [13]:
so2_ken["Parameter Name"].value_counts()

Sulfur dioxide    8845
Name: Parameter Name, dtype: int64

In [14]:
so2_ken["Units of Measure"].value_counts()

Parts per billion    8845
Name: Units of Measure, dtype: int64

In [15]:
so2_ken["Sample Duration"].value_counts()

1 HOUR          4423
3-HR BLK AVG    4422
Name: Sample Duration, dtype: int64

In [16]:
so2_ken["Pollutant Standard"].value_counts()

SO2 1-hour 2010    4423
SO2 3-hour 1971    4422
Name: Pollutant Standard, dtype: int64

In [17]:
so2_ken["AQI"].value_counts()

0.0      1970
1.0      1227
3.0       550
4.0       198
6.0       128
         ... 
94.0        1
57.0        1
82.0        1
120.0       1
116.0       1
Name: AQI, Length: 62, dtype: int64

# Merge with ah dataset

In [23]:
merged_ahs = pd.merge(merged_ah, so2_ken_f, how = "left", left_on = "County", right_on = "County Name",suffixes=('_ah', '_Sulfur'))
merged_ahs.head(2)

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,AQI_category
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
merged_ah.shape
merged_ahs.shape

(6900, 32)

(3334068, 62)

In [25]:
so2_ken_f.head(3)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,AQI_category
97694,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-01,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good
97695,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-02,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good
97696,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-03,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good


In [26]:
so2_ken_f.groupby(["AQI_category"])["County Name"].count()

AQI_category
Good                              4384
Moderate                            29
Unhealthy for Sensitive Groups      10
Name: County Name, dtype: int64

In [27]:
merged_ahs.groupby(["AQI_category"])["County Name"].count()

AQI_category
Good                              3326172
Moderate                             4260
Unhealthy for Sensitive Groups       1320
Name: County Name, dtype: int64

In [28]:
# your code here


##Visulation Part A (showing number of administrative identifier in each state)

#Groupby employer_state
#state_counts = so2_ken_f.groupby(['County Name',"AQI_category"]).count().reset_index().rename(columns={0: 'obs'})
#state_counts.head()

#p1 = go.Figure(data=go.Choropleth(
#    locations=["KY"],
#    z=state_counts['AQI_category'].astype("string"),
#    locationmode='country names',
#    colorscale='bluyl',
#    autocolorscale=False,
#    marker_line_color='black',
#    colorbar_title="Obs"
#))
#
#p1.update_layout(
#    title_text='Number of administrative identifier by states',
#    geo = dict(
#        scope='usa', # Confine map to US states and update layout
#        projection=go.layout.geo.Projection(type = 'albers usa'),
#       showlakes=True, ),
#)

In [29]:
merged_ahs.head(2)
merged_ahs.shape

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,AQI_category
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(3334068, 62)

In [43]:
merged_ahs["County"].value_counts()

Jefferson     2485200
Fayette        359160
Campbell       109200
Daviess        100188
McCracken       73440
Boyd            56940
Henderson       55968
Jessamine       39420
Greenup         39096
Edmonson        13140
Warren            288
Hardin            264
Boone             252
Pike              228
Christian         216
Bullitt           204
Oldham            168
Pulaski           168
Bell              108
Perry              96
Carter             84
Morgan             60
Simpson            48
Hancock            36
Trigg              36
Washington         36
Livingston         24
Name: County, dtype: int64

In [44]:
Greenup=merged_ahs[merged_ahs["County"]=="Greenup"]


In [45]:
Greenup.head()

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,AQI_category
639492,336,Kentucky,Greenup,2019,242,237,5,0,0,0,0,58,45,37,0,0,242,0,0,809750,2019,Greenup,21089,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.0,"3,120",POINT (-82.89613811 38.6581921),21089040501,CASTHMA,21.0,89.0,7.0,42401.0,1.0,38.548136,-82.731163,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-01,Parts per billion,None,23.0,96.0,0.043478,1.0,7.0,1.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,WORTHINGTON,SCOTT STREET & CENTER AVENUE,Kentucky,Greenup,Worthington,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good
639493,336,Kentucky,Greenup,2019,242,237,5,0,0,0,0,58,45,37,0,0,242,0,0,809750,2019,Greenup,21089,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.0,"3,120",POINT (-82.89613811 38.6581921),21089040501,CASTHMA,21.0,89.0,7.0,42401.0,1.0,38.548136,-82.731163,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-02,Parts per billion,None,23.0,96.0,0.000000,0.0,0.0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,WORTHINGTON,SCOTT STREET & CENTER AVENUE,Kentucky,Greenup,Worthington,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good
639494,336,Kentucky,Greenup,2019,242,237,5,0,0,0,0,58,45,37,0,0,242,0,0,809750,2019,Greenup,21089,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.0,"3,120",POINT (-82.89613811 38.6581921),21089040501,CASTHMA,21.0,89.0,7.0,42401.0,1.0,38.548136,-82.731163,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-03,Parts per billion,None,23.0,96.0,0.086957,1.0,5.0,1.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,WORTHINGTON,SCOTT STREET & CENTER AVENUE,Kentucky,Greenup,Worthington,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good
639495,336,Kentucky,Greenup,2019,242,237,5,0,0,0,0,58,45,37,0,0,242,0,0,809750,2019,Greenup,21089,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.0,"3,120",POINT (-82.89613811 38.6581921),21089040501,CASTHMA,21.0,89.0,7.0,42401.0,1.0,38.548136,-82.731163,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-04,Parts per billion,None,23.0,96.0,0.086957,1.0,15.0,1.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,WORTHINGTON,SCOTT STREET & CENTER AVENUE,Kentucky,Greenup,Worthington,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good
639496,336,Kentucky,Greenup,2019,242,237,5,0,0,0,0,58,45,37,0,0,242,0,0,809750,2019,Greenup,21089,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.0,"3,120",POINT (-82.89613811 38.6581921),21089040501,CASTHMA,21.0,89.0,7.0,42401.0,1.0,38.548136,-82.731163,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-05,Parts per billion,None,22.0,92.0,0.000000,0.0,0.0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,WORTHINGTON,SCOTT STREET & CENTER AVENUE,Kentucky,Greenup,Worthington,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good


In [30]:
#!pip install geopandas==0.8.1 
#!pip install pyshp==1.2.10
#!pip install shapely

In [31]:
#!pip install pyshp

In [32]:
Plot_df = pd.DataFrame(merged_ahs[['AQI','CountyFIPS']])

In [33]:
Plot_df_n = Plot_df.dropna(subset=["AQI"],how='all',inplace=False).reset_index()

In [34]:
Plot_df.shape
Plot_df_n.shape

(3334068, 2)

(3331752, 3)

In [35]:
Plot_df_na = Plot_df.dropna(subset=["CountyFIPS"],how='all',inplace=False).reset_index()

In [36]:
Plot_df_na.shape

(3334068, 3)

In [47]:
Plot_df_greenup = pd.DataFrame(Greenup[['AQI','CountyFIPS']]).dropna(subset=["AQI"],how='all',inplace=False).reset_index()


In [49]:
Plot_df_na_greenup = Plot_df_greenup.dropna(subset=["CountyFIPS"],how='all',inplace=False).reset_index()

In [54]:
Plot_df_na_greenup.head()

,level_0,index,AQI,CountyFIPS
0,0,639492,1.0,21089
1,1,639493,0.0,21089
2,2,639494,1.0,21089
3,3,639495,1.0,21089
4,4,639496,0.0,21089


In [37]:
import plotly.express as px
import requests
import json
import pandas as pd


In [50]:
r = requests.get('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
counties = json.loads(r.text)
target_states = ['21']
counties['features'] = [f for f in counties['features'] if f['properties']['STATE'] in target_states]

#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv', dtype={'fips': str})



In [53]:
fig = px.choropleth(Plot_df_na_greenup, geojson=counties, locations='CountyFIPS', color='AQI',
                    color_continuous_scale='Viridis',
                    range_color=(0, 100),
                    scope='usa'
                    
                    )
#fig.update_layout(margin={'r': 0, 't': 0, 'l': 0, 'b': 0})
#labels={'unemp': 'unemployment rate'}
fig.show()

In [52]:
#endpts = list(np.mgrid[min(values):max(values):4j])
#colorscale = ["#030512","#1d1d3b","#323268","#3d4b94","#3e6ab0",
#              "#4989bc","#60a7c7","#85c5d3","#b7e0e4","#eafcfd"]
#fig = ff.create_choropleth(
#    fips=Plot_df_na["CountyFIPS"], values=Plot_df_na["AQI"], scope=['Kentucky'], show_state_data=True,
#    )
#fig.layout.template = None
#fig.show()

In [ ]:
#df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/minoritymajority.csv')
#df_sample_r = df_sample[df_sample['STNAME'] == 'Florida']

#values = merged_ahs['AQI'].tolist()
#fips = merged_ahs['CountyFIPS'].tolist()

#value_n = [item for item in values if not(math.isnan(item)) == True]
 

#endpts = list(np.mgrid[min(values):max(values):4j])
#colorscale = ["#030512","#1d1d3b","#323268","#3d4b94","#3e6ab0",
#              "#4989bc","#60a7c7","#85c5d3","#b7e0e4","#eafcfd"]
#fig = ff.create_choropleth(
#    fips=fips, values=values, scope=['Florida'], show_state_data=True,
#    colorscale=colorscale, binning_endpoints=endpts, round_legend_values=True,
#    plot_bgcolor='rgb(229,229,229)',
#    paper_bgcolor='rgb(229,229,229)',
#    legend_title='Population by County',
#    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
#    exponent_format=True,
#)
#fig.layout.template = None
#fig.show()

In [55]:
merged_ahs.head()

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,AQI_category
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809818,2019,Bell,21013,Health Risk Behaviors,Current smoking among adults aged >=18 years,%,Crude prevalence,33.3,"3,139",POINT (-83.81829927 36.66805338),21013961100,CSMOKING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809827,2019,Bell,21013,Prevention,Visits to doctor for routine checkup within th...,%,Crude prevalence,80.7,"4,747",POINT (-83.76654247 36.59758829),21013960600,CHECKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,809884,2019,Bell,21013,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,11.8,"3,771",POINT (-83.76484451 36.64167333),21013960800,CASTHMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
health_ken_2019_fm.head()

,index,Year,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
0,3596,2019,Adair,21001,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,44.8,"3,016",POINT (-85.15821669 37.13121962),21001970300,LPA
2,3598,2019,Anderson,21005,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,26.6,"5,372",POINT (-84.92306901 38.02049683),21005950201,DEPRESSION
11,3607,2019,Casey,21045,Prevention,Taking medicine for high blood pressure contro...,%,Crude prevalence,80.0,"2,370",POINT (-85.06611177 37.33858656),21045950200,BPMED
13,3609,2019,Clark,21049,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,11.8,"3,107",POINT (-84.18804198 38.02446147),21049020201,CASTHMA
17,3613,2019,Clay,21051,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,29.8,"2,661",POINT (-83.79297905 37.08231102),21051950500,DEPRESSION


In [84]:
health_ken_2019_fm.shape

(13272, 13)

In [57]:
health_ken_2019_fm["Data_Value_Type"].value_counts()

Crude prevalence    13272
Name: Data_Value_Type, dtype: int64

In [69]:
df_plot1=pd.DataFrame(health_ken_2019_fm.groupby(["Category","CountyName"])["Data_Value"].mean())
df_plot1 = df_plot1.reset_index()

In [70]:
df_plot1

,Category,CountyName,Data_Value
0,Health Outcomes,Adair,14.503571
1,Health Outcomes,Allen,14.200000
2,Health Outcomes,Anderson,13.455000
3,Health Outcomes,Ballard,14.383333
4,Health Outcomes,Barren,14.587500
...,...,...,...
475,Prevention,Wayne,58.060000
476,Prevention,Webster,57.583333
477,Prevention,Whitley,56.691667
478,Prevention,Wolfe,58.150000


In [63]:
import matplotlib.pyplot as plt

%matplotlib inline

In [72]:
import plotly.graph_objects as go